In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()

23/07/10 08:27:51 WARN Utils: Your hostname, Benhards-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.14 instead (on interface en0)
23/07/10 08:27:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/10 08:27:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/10 08:27:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/10 08:27:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/07/10 08:27:53 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [12]:
training = spark.read.csv('data.csv',header = True, inferSchema = True)
training.show()
training.printSchema()
training.columns

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|Michael| 21|        10| 10000|
|  Karen| 22|         8|  5000|
| Jessie| 20|         4|  4000|
|   Jena| 24|         3|  4000|
|   Paul| 19|         1|  3000|
|  Jones| 30|         2| 20000|
+-------+---+----------+------+

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



['Name', 'Age', 'Experience', 'Salary']

In [13]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Feature")

In [15]:
output = featureassembler.transform(training)
output.show()

+-------+---+----------+------+-------------------+
|   Name|Age|Experience|Salary|Independent Feature|
+-------+---+----------+------+-------------------+
|Michael| 21|        10| 10000|        [21.0,10.0]|
|  Karen| 22|         8|  5000|         [22.0,8.0]|
| Jessie| 20|         4|  4000|         [20.0,4.0]|
|   Jena| 24|         3|  4000|         [24.0,3.0]|
|   Paul| 19|         1|  3000|         [19.0,1.0]|
|  Jones| 30|         2| 20000|         [30.0,2.0]|
+-------+---+----------+------+-------------------+



In [17]:
finalized_data = output.select("Independent Feature", "Salary")
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [21.0,10.0]| 10000|
|         [22.0,8.0]|  5000|
|         [20.0,4.0]|  4000|
|         [24.0,3.0]|  4000|
|         [19.0,1.0]|  3000|
|         [30.0,2.0]| 20000|
+-------------------+------+



In [25]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent Feature', labelCol = 'Salary')
regressor = regressor.fit(train_data)

23/07/10 08:45:49 WARN Instrumentation: [017dc7cc] regParam is zero, which might cause numerical instability and overfitting.


In [26]:
regressor.coefficients

DenseVector([-121.9512, 804.878])

In [27]:
regressor.intercept

4512.19512195114

In [29]:
pred_result = regressor.evaluate(test_data)
pred_result.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [20.0,4.0]|  4000|5292.682926829264|
|         [22.0,8.0]|  5000| 8268.29268292683|
|         [30.0,2.0]| 20000|2463.414634146377|
+-------------------+------+-----------------+



In [30]:
pred_result.meanAbsoluteError, pred_result.meanSquaredError

(7365.853658536573, 106628197.50148678)